In [5]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import numpy as np
import plotly.graph_objects as go
from scipy.spatial import ConvexHull
import pandas as pd

# Generate random 3D points with labels
np.random.seed(42)
x = np.random.randn(100)
y = np.random.randn(100)
z = np.random.randn(100)

# Assign random labels
labels = np.random.choice([0, 1, 2, 3], size=100)

# df = pd.DataFrame(np.append(np.random.randn(100,3), labels), columns=["x","y","z","label"])
df = pd.DataFrame(np.random.randn(100,3), columns=["x", "y", "z"])

X = df.values

df["label"] = labels


In [4]:
# Colors for the groups
colors = ['lightblue', 'lightgreen', 'lightpink', 'red']

# Create the Dash app
app = dash.Dash(__name__)

# Create a 3D-hull scatter plot given covariates X (n x 3) and labels y (n x 1)
def create_figure(X, y):

    # Array containing all the clusters and their convex hulls
    layers = []
    
    # Create scatter plots and convex hull volumes for each label in y
    for label in np.unique(y):
        
        # select rows with corresponding label
        X_k = X[y==label]

        # Compute the convex hull for this group if it has enough points
        if X_k.shape[0] >= 4:
            hull = ConvexHull(X_k)

            x_c, y_c, z_c  = X_k[:,0], X_k[:,1], X_k[:,2] 

            # Scatter plot for points
            scatter = go.Scatter3d(
                x=x_c, y=y_c, z=z_c, 
                mode='markers', 
                marker=dict(size=5, color=colors[label], opacity=0.8),
                name=f'Label {label}',
                legendgroup=f'Label {label}',
                showlegend=False,
            )
            layers.append(scatter)

            # Mesh3d for convex hull
            mesh = go.Mesh3d(
                x=x_c,
                y=y_c,
                z=z_c,
                i=hull.simplices[:, 0],
                j=hull.simplices[:, 1],
                k=hull.simplices[:, 2],
                opacity=0.3,
                color=colors[label],
                hoverinfo='text',
                hovertext=f'Hull enclosing Label {label} points',
                name=f'Hull {label}',
                legendgroup=f'Hull {label}'
            )
            layers.append(mesh)

    # Create figure
    fig = go.Figure(data=layers)
    fig.update_layout(plot_bgcolor='lightblue',
                      paper_bgcolor='darkgray',
                      scene=dict(aspectmode='data',
                                xaxis=dict(visible=False),
                                yaxis=dict(visible=False),
                                zaxis=dict(visible=False)),
                      title="Job Space",
                    )

    return fig

# Layout of the app
app.layout = html.Div([
    dcc.Graph(id='3d-scatter', figure=create_figure(X, labels)),
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
